In [3]:
#1
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd


df = pd.read_csv("ZIONO_weekly_return_volatility.csv")
df.label = df.label.fillna(method='ffill')
df_21 = df[df.Year==2021]
df_22 = df[df.Year==2022]
df_22.head(10)
df_22.head() # mean

,Year,Week_Number,mean_return,volatility,label
54,2022,1,-0.2012,0.842329,red
55,2022,2,0.1002,0.604448,green
56,2022,3,0.1525,0.591757,green
57,2022,4,-0.3576,1.847664,red
58,2022,5,-0.1206,0.762250,red


In [7]:
# 1 , construct df_train
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from scipy import stats

x_21 = df_21[["mean_return", "volatility"]].values
le = LabelEncoder()
y_21 = le.fit_transform(df_21["label"].values)
y_22 = le.transform(df_22["label"].values)


df_train = pd.DataFrame({'x1': df_21["mean_return"].values, 'x2': df_21['volatility'], 'y': y_21})
df_train.head()

,x1,x2,y
2,-0.2586,0.729467,0
3,0.0776,0.787634,1
4,0.9710,2.437100,0
5,-0.5042,1.114784,1
6,0.1458,1.217830,0


In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import sklearn.svm as svm

X = df_train[[ 'x1', 'x2']]. values
scaler = StandardScaler(). fit (X)
X = scaler.transform (X)
Y = df_train['y']. values


# 建立模型
model_line_svm = svm.SVC(C=10, kernel='linear')

# 训练模型,x为训练集， y为标签
model_line_svm.fit(X, Y)

new_x = scaler.transform(df_22[["mean_return", "volatility"]].values)
predicted = model_line_svm.predict ( new_x )
line_predict = predicted
predicted

array([1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0])

In [11]:
y_22

array([1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1])

In [13]:
from sklearn.metrics import accuracy_score

accuracy_score(y_22, predicted)

0.9423076923076923

In [14]:
# 2
from sklearn.metrics import confusion_matrix

confusion_matrix(y_22, predicted)

array([[25,  2],
       [ 1, 24]], dtype=int64)

In [17]:
# for quiz
rec= 24/25
prec = 24 / 26

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
recall_score(y_22, predicted), precision_score(y_22, predicted), rec, prec

(0.96, 0.9230769230769231, 0.96, 0.9230769230769231)

In [20]:
# 3 tnr / tpr
tn, fp, fn, tp = confusion_matrix(y_22, predicted).ravel()

tnr = tn / (tn + fp)
tpr = tp / (tp + fn)

tnr, tpr

(0.9259259259259259, 0.96)

In [24]:
# 4 Gaussian
# 建立模型
model_rbf_svm = svm.SVC(C=10, kernel='rbf')

# 训练模型,x为训练集， y为标签
model_rbf_svm.fit(X, Y)

new_x = scaler.transform(df_22[["mean_return", "volatility"]].values)
predicted = model_rbf_svm.predict ( new_x )
predicted

accuracy_score(y_22, predicted)

# 0.9423076923076923

0.8846153846153846

In [25]:
# 4---  worse than linear


In [27]:
# 5 ---- poly
# 建立模型
model_poly_svm = svm.SVC(C=10, kernel='poly', degree=2)

# 训练模型,x为训练集， y为标签
model_poly_svm.fit(X, Y)

new_x = scaler.transform(df_22[["mean_return", "volatility"]].values)
predicted = model_poly_svm.predict ( new_x )
predicted

accuracy_score(y_22, predicted)

0.4807692307692308

In [28]:
# 5 worse than linear

In [32]:
# 6. 
# buy and_hold: week1   25,,,--> week52  27---> + 2
# predict.....:
# line_predict
df_detail = pd.read_csv("ZIONO_weekly_return_detailed.csv")
df_detail.head(22)
df_detail_22 = df_detail[["Close", "Week_Number", "Year"]]
df_detail_22 = df_detail_22[df_detail_22.Year == 2022]
df_22_week = df_detail_22.groupby("Week_Number").last()
df_22_week.head()

,Close,Year
Week_Number,,
1,26.290001,2022
2,26.420000,2022
3,26.580000,2022
4,26.090000,2022
5,25.930000,2022


In [34]:
 df_22_week.tail()

,Close,Year
Week_Number,,
48,25.180000,2022
49,25.129999,2022
50,25.299999,2022
51,25.120001,2022
52,25.370001,2022


In [36]:
# low, ----  buy_and_hold
buy_hold_add = df_22_week.Close.values[-1] - df_22_week.Close.values[0]
buy_hold_add

-0.9200000762939453

In [41]:
# mine---
line_predict, df_22_week.values


# my strategy
def get_benefit_my_strategy(pred_labels, close_prices):
    money = 100
    hold_share = False
    for i in range(len(pred_labels)): # 1 red
        if (pred_labels[i] == 1): # red---> if not owning, buy one
            if hold_share:
                pass
            else:
                hold_share = True
                money -= close_prices[i]
        else: # green ---->if owning, sell it
            if hold_share:
                hold_share = False
                money += close_prices[i]
    share_value = close_prices[-1] if hold_share else 0
    return money + share_value - 100
get_benefit_my_strategy( line_predict.tolist(), df_22_week.Close.values.tolist())

1.7599964141845703

In [ ]:
# our stratedy win!!!!!